In [1]:
import matplotlib.pyplot as plt

import pypsa

%matplotlib inline
plt.rc("figure", figsize=(8, 8))

In [2]:
network = pypsa.examples.ac_dc_meshed(from_master=True)

INFO:pypsa.io:Imported network ac-dc-meshed.nc has buses, carriers, generators, global_constraints, lines, links, loads


In [12]:
# get current type (AC or DC) of the lines from the buses
lines_current_type = network.lines.bus0.map(network.buses.carrier)
network.buses

,v_nom,carrier,x,y,type,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
London,380.0,AC,-0.13,51.50,,,1.0,0.0,inf,PQ,,0
Norwich,380.0,AC,1.30,52.60,,,1.0,0.0,inf,PQ,,0
Norwich DC,200.0,DC,1.30,52.50,,,1.0,0.0,inf,Slack,None,1
Manchester,380.0,AC,-2.20,53.47,,,1.0,0.0,inf,Slack,Manchester Wind,0
Bremen,380.0,AC,8.80,53.08,,,1.0,0.0,inf,PQ,,2
Bremen DC,200.0,DC,8.80,52.98,,,1.0,0.0,inf,PQ,,1
Frankfurt,380.0,AC,8.70,50.12,,,1.0,0.0,inf,Slack,Frankfurt Wind,2
Norway,380.0,AC,10.75,60.00,,,1.0,0.0,inf,Slack,Norway Wind,3
Norway DC,200.0,DC,10.75,60.00,,,1.0,0.0,inf,PQ,,1


In [14]:
network.lines

,bus0,bus1,capital_cost,s_nom,s_nom_extendable,x,r,type,g,b,...,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,v_nom
Line,,,,,,,,,,,,,,,,,,,,,
0,London,Manchester,0.136716,40000.0,True,0.796878,0.000000,,0.0,0.0,...,inf,0,0.000006,0.000000,0.0,0.0,0.000006,0.000000,0.0,380.0
1,Manchester,Norwich,0.133492,40000.0,True,0.391560,0.000000,,0.0,0.0,...,inf,0,0.000003,0.000000,0.0,0.0,0.000003,0.000000,0.0,380.0
2,Bremen DC,Norwich DC,0.008673,40000.0,True,0.000000,0.212604,,0.0,0.0,...,inf,1,0.000000,0.000005,0.0,0.0,0.000000,0.000005,0.0,200.0
3,Norwich DC,Norway DC,0.129126,40000.0,True,0.000000,0.486164,,0.0,0.0,...,inf,1,0.000000,0.000012,0.0,0.0,0.000000,0.000012,0.0,200.0
4,Norway DC,Bremen DC,0.062430,40000.0,True,0.000000,0.428727,,0.0,0.0,...,inf,1,0.000000,0.000011,0.0,0.0,0.000000,0.000011,0.0,200.0
5,Norwich,London,0.021852,40000.0,True,0.238800,0.000000,,0.0,0.0,...,inf,0,0.000002,0.000000,0.0,0.0,0.000002,0.000000,0.0,380.0
6,Bremen,Frankfurt,0.200000,40000.0,True,0.400000,0.000000,,0.0,0.0,...,inf,2,0.000003,0.000000,0.0,0.0,0.000003,0.000000,0.0,380.0


In [23]:
network.links.keys()

Index(['bus0', 'bus1', 'capital_cost', 'p_max_pu', 'p_min_pu', 'p_nom',
       'p_nom_extendable', 'type', 'carrier', 'efficiency', 'build_year',
       'lifetime', 'p_nom_mod', 'p_nom_min', 'p_nom_max', 'p_set',
       'marginal_cost', 'marginal_cost_quadratic', 'stand_by_cost', 'length',
       'terrain_factor', 'committable', 'start_up_cost', 'shut_down_cost',
       'min_up_time', 'min_down_time', 'up_time_before', 'down_time_before',
       'ramp_limit_up', 'ramp_limit_down', 'ramp_limit_start_up',
       'ramp_limit_shut_down', 'p_nom_opt'],
      dtype='object')

In [5]:
network.links.loc["Norwich Converter", "p_nom_extendable"] = False

In [6]:
network.determine_network_topology()
network.sub_networks["n_branches"] = [
    len(sn.branches()) for sn in network.sub_networks.obj
]
network.sub_networks["n_buses"] = [len(sn.buses()) for sn in network.sub_networks.obj]

network.sub_networks

attribute,carrier,slack_bus,obj,n_branches,n_buses
SubNetwork,,,,,
0,AC,Manchester,SubNetwork 0,3,3
1,DC,Norwich DC,SubNetwork 1,3,3
2,AC,Frankfurt,SubNetwork 2,1,2
3,AC,Norway,SubNetwork 3,0,1


In [19]:
network.optimize()

Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
Index(['2', '3', '4'], dtype='object', name='Line')
Index(['0', '1', '5', '6'], dtype='object', name='Line')
INFO:linopy.model: Solve problem using Glpk solver


AssertionError: Solver glpk not installed

In [ ]:
network.objective

In [ ]:
network.objective + network.objective_constant

In [ ]:
network.generators.p_nom_opt.div(1e3).plot.bar(ylabel="GW", figsize=(8, 3))
plt.tight_layout()

In [ ]:
network.generators_t.p.div(1e3).plot.area(subplots=True, ylabel="GW")
plt.tight_layout()

In [ ]:
network.buses_t.marginal_price.mean(1).plot.area(figsize=(8, 3), ylabel="Euro per MWh")
plt.tight_layout()

In [ ]:
network.links_t.p0

In [ ]:
network.lines_t.p0

In [ ]:
network.buses_t.p